In [1]:
import numpy as np
import pandas as pd

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import ast
from joblib import dump, load

In [2]:
df_train = pd.read_csv('database/LaA_train.csv')
df_test = pd.read_csv('database/LaA_test.csv')
df_train.head()

,Unnamed: 0,artist_name,track_name,valence,arousal,lyrics,audio_url,search_method,emotion,lyrics_embedding,audio_embedding,label
0,6481,Alexz Johnson,White Lines,0.678952,-2.333604,I tried to tell you\nI've got to get away\nI t...,https://p.scdn.co/mp3-preview/eae97329ac7135a5...,artist_and_song,Relaxed,"[0.5272476077079773, 1.168580174446106, 0.1721...","[0.7431232416583579, 0.20046921661336187, 0.23...",2
1,9075,Turntablerocker,No Melody,0.373325,-0.923151,We've got the song\nBut they got no melody\nNo...,https://p.scdn.co/mp3-preview/4f397176ee912edf...,artist_and_song,Relaxed,"[-0.5302870273590088, -1.6064175367355347, 1.8...","[2.089026585978773, 1.0356926605543044, 0.7353...",2
2,9082,Lamb,Zero,-0.367547,-0.939283,LAMB: ZERO\n\nThere's no one here today\n'Caus...,https://p.scdn.co/mp3-preview/f65a72ecfbf7c304...,artist_and_song,Sad,"[0.054062437266111374, 0.38927221298217773, -0...","[0.8801736065962609, 1.9399530531306048, 2.596...",3
3,10587,Einstuerzende Neubauten,Youme & Meyou,0.526139,-1.628377,They build a ship each wintertime\nFor launch ...,https://p.scdn.co/mp3-preview/a2acca0ac29d3a0a...,artist_and_song,Relaxed,"[-1.2014904022216797, -0.5136592388153076, -0....","[0.5157966639886593, 1.5658099646756507, 1.259...",2
4,9177,Mouse On Mars,Wipe That Sound,0.815393,0.662457,"Kick the can\nI kick, kick kick the can\nI kic...",https://p.scdn.co/mp3-preview/a803171c426e144d...,artist_and_song,Happy,"[-0.42865195870399475, 0.8299823999404907, 1.1...","[2.3368670630266033, 1.3798174751404781, 0.647...",1


In [3]:
embeddings_audio_train = df_train['audio_embedding'].apply(ast.literal_eval).apply(np.array)
x_audio_train = np.stack(embeddings_audio_train.values)
embeddings_audio_test = df_test['audio_embedding'].apply(ast.literal_eval).apply(np.array)
x_audio_test = np.stack(embeddings_audio_test.values)

embeddings_lyrics_train = df_train['lyrics_embedding'].apply(ast.literal_eval).apply(np.array)
x_lyrics_train = np.stack(embeddings_lyrics_train.values)
embeddings_lyrics_test = df_test['lyrics_embedding'].apply(ast.literal_eval).apply(np.array)
x_lyrics_test = np.stack(embeddings_lyrics_test.values)

y_train = df_train['label']
y_test = df_test['label']




#### Either train SVMs again or load it 
if loaded watch out that same data split and same datascaler, etc.

In [ ]:

"""# Initialize and train the SVM classifier on the scaled dataset
svm_classifier_lyrics = SVC(kernel='rbf', C=1, gamma='auto', probability=True)
svm_classifier_lyrics.fit(x_lyrics_train, y_train)
print("Lyrics SVM trained")
svm_classifier_audio = SVC(kernel='rbf', C=1, gamma='auto', probability=True)
svm_classifier_audio.fit(x_audio_train, y_train)
print("Audio SVM trained")"""


In [4]:
name_svm_audio = 'models/SVM_audio.joblib'
name_svm_lyrics = 'models/SVM_lyrics.joblib'
svm_classifier_audio = load(name_svm_audio)
svm_classifier_lyrics = load(name_svm_lyrics)


In [12]:
# Predict the probabilities
y_pred_test_prob_lyrics = svm_classifier_lyrics.predict_proba(x_lyrics_test)
y_pred_test_prob_audio = svm_classifier_audio.predict_proba(x_audio_test)
y_pred_train_prob_lyrics = svm_classifier_lyrics.predict_proba(x_lyrics_train)
y_pred_train_prob_audio = svm_classifier_audio.predict_proba(x_audio_train)

In [8]:
#get labels
y_pred_lyrics = np.argmax(y_pred_test_prob_lyrics, axis=1)
y_pred_audio = np.argmax(y_pred_test_prob_audio, axis=1)

acc_lyrics, acc_audio = accuracy_score(y_test, y_pred_lyrics), accuracy_score(y_test, y_pred_audio)
print(f"Lyrics accuracy: {acc_lyrics:.2f}")
print(f"Audio accuracy: {acc_audio:.2f}")

Lyrics accuracy: 0.43
Audio accuracy: 0.42


#### First kind of models take probabilities and are simple

In [9]:
#First model takes the highest probability of the two added models
y_pred_max_combined = np.argmax(y_pred_test_prob_lyrics + y_pred_test_prob_audio, axis=1)
acc_max_combined = accuracy_score(y_test, y_pred_max_combined)
print(f"Accuracy: {acc_max_combined:.2f}")

V1 accuracy: 0.45


In [10]:

#Second model takes the higher probability of the model
y_pred_max = np.argmax(np.maximum(y_pred_test_prob_lyrics, y_pred_test_prob_audio), axis=1)
acc_max = accuracy_score(y_test, y_pred_max)
print(f"Accuracy: {acc_max:.2f}")


V2 accuracy: 0.45


#### Second kind of models take probabilities as new input

In [14]:
x_meta_train = np.hstack((y_pred_train_prob_lyrics, y_pred_train_prob_audio))
x_meta_test = np.hstack((y_pred_test_prob_lyrics, y_pred_test_prob_audio))

In [15]:
from sklearn.ensemble import RandomForestClassifier

meta_classifier_forest = RandomForestClassifier()
meta_classifier_forest.fit(x_meta_train, y_train)
y_pred_forest = meta_classifier_forest.predict(x_meta_test)
acc_forest = accuracy_score(y_test, y_pred_forest)
print(f"Accuracy: {acc_forest:.2f}")



V3 accuracy: 0.43


In [16]:
from sklearn.linear_model import LogisticRegression

# Using the same meta features as before
meta_classifier_lr = LogisticRegression(max_iter=1000)
meta_classifier_lr.fit(x_meta_train, y_train)

# Predictions
y_pred_lr = meta_classifier_lr.predict(x_meta_test)
acc_meta_lr = accuracy_score(y_test, y_pred_lr)
print(f"Accuracy: {acc_meta_lr:.2f}")


Accuracy: 0.45


In [18]:
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

x_meta_train_tensor = torch.tensor(x_meta_train, dtype=torch.float)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
x_meta_test_tensor = torch.tensor(x_meta_test, dtype=torch.float)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

train_dataset = TensorDataset(x_meta_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataset = TensorDataset(x_meta_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

class MetaClassifierNN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(MetaClassifierNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)  # First hidden layer
        self.fc2 = nn.Linear(128, 64)  # Second hidden layer
        self.fc3 = nn.Linear(64, num_classes)  # Output layer
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize the model
model = MetaClassifierNN(input_size=8, num_classes=4)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 20
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        optimizer.zero_grad()  # Clear gradients for this training step
        outputs = model(inputs)  # Forward pass
        loss = criterion(outputs, labels)  # Calculate loss
        loss.backward()  # Backward pass
        optimizer.step()  # Apply gradients
    
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

Epoch 1/20, Loss: 0.7354363799095154
Epoch 2/20, Loss: 0.5320879817008972
Epoch 3/20, Loss: 0.5591878890991211
Epoch 4/20, Loss: 0.515727162361145
Epoch 5/20, Loss: 0.6375213861465454
Epoch 6/20, Loss: 0.7026153206825256
Epoch 7/20, Loss: 0.46629008650779724
Epoch 8/20, Loss: 0.7474197745323181
Epoch 9/20, Loss: 0.6568333506584167
Epoch 10/20, Loss: 0.4438040554523468
Epoch 11/20, Loss: 0.8439077734947205
Epoch 12/20, Loss: 0.5823493599891663
Epoch 13/20, Loss: 0.389651358127594
Epoch 14/20, Loss: 0.7707103490829468
Epoch 15/20, Loss: 0.3900233507156372
Epoch 16/20, Loss: 0.6029807925224304
Epoch 17/20, Loss: 0.580782413482666
Epoch 18/20, Loss: 0.7533096075057983
Epoch 19/20, Loss: 0.6467638611793518
Epoch 20/20, Loss: 0.743520975112915


In [19]:
model.eval()

correct = 0
total = 0
with torch.no_grad(): 
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy_NN = correct / total
print(f'Accuracy on the test set: {accuracy_NN * 100:.2f}%')

Accuracy on the test set: 43.64%
